In [1]:
import cv2
import time 
import numpy as np
from matplotlib import pyplot as plt
import os

In [2]:
path_ =  './train/'

In [3]:
# print('СПИСОК НАЗВАНИЙ ВИДЕО ДЛЯ ОБУЧЕНИЯ\n')
# for video in os.listdir(path_):
#     print(video)

СПИСОК НАЗВАНИЙ ВИДЕО ДЛЯ ОБУЧЕНИЯ

видео по 1 шаблону пульс 90-80.MOV
ant.mp4
пульс 90-80 видео по 2 шаблону.MOV
спокойное дыхание пульс 90.MOV
me_train.mov
nikita.mp4
пульс 116-90.MOV
видос по 2 шаблону.MOV
nekit_train.mov
пульс 160-150 4 шаблон.MOV
видео по 4 шаблону пульс 154.MOV
спокойное дыхание по 1 шаблону.MOV
пульс 142-108 3-4 шаблон.MOV
test.mp4


In [11]:
path_ =  './train/'
file_ = 'пульс 142-108 3-4 шаблон.MOV'

addr_ = path_ + file_

In [4]:
def rr(cr_im):
    fl = 0
    try:
        heartrate = int(np.average(cr_im))
        if heartrate not in range(50, 101):
        # RR is NOT okay
            fl = 1
            return fl
        else:
            # RR is okay
            return fl
    except:
        return fl

In [8]:
def detect_face(frame):
    cascPath = "./cascades/haarcascade_frontalface_default.xml"
    faceCascade = cv2.CascadeClassifier(cascPath)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the image
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=7.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE
    )
    x, y, w, h = 0, 0, 0, 0
    # Draw a rectangle around the faces
    for (x, y, w, h) in faces:
        cv2.rectangle(gray, (x, y), (x + w, y + h), (0, 255, 0), 2)

    return (x, y, w, h)

In [12]:
#### BLUR FACE ####
def blur_face(addr):
    cap = cv2.VideoCapture(addr)

    heartbeat_count = 128
    heartbeat_values = [0]*heartbeat_count
    heartbeat_times = [time.time()]*heartbeat_count
    breath_rate = [0]*heartbeat_count

    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax2 = fig.add_subplot(111)

    count_frame = 0
    
    # for the fist frame
    status, frame = cap.read()
    coords = detect_face(frame)
    x, y, w, h = coords[0], coords[1], coords[2], coords[3]

    while True:
        status, frame = cap.read()
        
        try:
            if frame.all() == None:
                print('video has ended')
        except:
            print('eof')
            break
            
        frame=cv2.flip(frame,1)
        kernel = np.ones((3, 3),np.uint8)

        count_frame+=1
        
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        if count_frame%15==0: 
            
            img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            coords = detect_face(frame)
            x, y, w, h = coords[0], coords[1], coords[2], coords[3]

        crop_img = img[y:y + h, x:x + w]
        try:
            crop_img = cv2.dilate(crop_img, kernel, iterations = 3)
            crop_img = cv2.GaussianBlur(crop_img,(3,3),1)
        except:
            pass

        heartbeat_values = heartbeat_values[1:] + [np.average(crop_img)]
        heartbeat_times = heartbeat_times[1:] + [time.time()]
        rr_ = rr(crop_img)
        breath_rate = breath_rate[1:] + [rr_]
    #     breath_rate.append(rr_plot(np.average(crop_img)))

            # Draw matplotlib graph to numpy array
        ax.plot(heartbeat_times, heartbeat_values)
        fig.canvas.draw()
        plot_img_np = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
        plot_img_np = plot_img_np.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        plt.cla()

        #draw rr plot
        ax2.plot(heartbeat_times, breath_rate)
        fig.canvas.draw()
        rr_plot = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
        rr_plot = rr_plot.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        plt.cla()

        # Display the frames
        try:
            cv2.imshow('Face', crop_img)
        except:
            pass

        cv2.imshow('HR', plot_img_np)
        cv2.imshow('RR', rr_plot)

        if cv2.waitKey(1) & 0xFF == ord('z'):
            break
            
    cap.release()
    cv2.destroyAllWindows()

In [15]:
# по обучающему видео

# blur_face(addr_)

In [16]:
# по тестовому видео
path_test =  './Для теста/'
addr_test = path_test + os.listdir(path_test)[0]

blur_face(addr_test)

<ipython-input-12-0e6813483c3d>:12: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  ax2 = fig.add_subplot(111)
c:\program files\python38\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
c:\program files\python38\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-12-0e6813483c3d>:60: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  plot_img_np = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
<ipython-input-12-0e6813483c3d>:67: DeprecationWarnin

Traceback (most recent call last):
  File "c:\program files\python38\lib\site-packages\IPython\core\interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-16-32ee601af2d9>", line 5, in <module>
    blur_face(addr_test)
  File "<ipython-input-12-0e6813483c3d>", line 41, in blur_face
    coords = detect_face(frame)
  File "<ipython-input-8-3b732db376a7>", line 3, in detect_face
    faceCascade = cv2.CascadeClassifier(cascPath)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\program files\python38\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\program files\python38\lib\gener

TypeError: object of type 'NoneType' has no len()

Error in callback <function flush_figures at 0x000001F5E0FA9D30> (for post_execute):


KeyboardInterrupt: 

In [17]:
cv2.destroyAllWindows()